<a href="https://colab.research.google.com/github/LivesayMe/DeltaMusic/blob/main/ImagineDragonsGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@test {"output": "ignore"}
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow
import pretty_midi

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Installing dependencies...
Importing libraries and defining some helper functions...
🎉 Done!
2.1.3
2.4.1


In [6]:
from note_seq.protobuf import music_pb2

curSong = note_seq.midi_file_to_sequence_proto('/content/MIDI Data/Demons.mid')
note_seq.plot_sequence(curSong)
note_seq.play_sequence(curSong,synth=note_seq.fluidsynth)

In [9]:
%%shell
convert_dir_to_note_sequences \
  --input_dir=/content/"MIDI Data" \
  --output_file=tmp/notesequences.tfrecord \
  --recursive

2021-02-09 16:47:32.780739: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Converting files in '/content/MIDI Data/'.
I0209 16:47:38.338353 139811074758528 convert_dir_to_note_sequences.py:83] Converting files in '/content/MIDI Data/'.
INFO:tensorflow:0 files converted.
I0209 16:47:38.339650 139811074758528 convert_dir_to_note_sequences.py:89] 0 files converted.
INFO:tensorflow:Converted MIDI file /content/MIDI Data/Radioactive.mid

In [11]:
%%shell
polyphony_rnn_create_dataset \
--input=tmp/notesequences.tfrecord \
--output_dir=tmp/polyphony_rnn/sequence_examples \
--eval_ratio=0.10

2021-02-09 16:48:14.206102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0209 16:48:18.302738 140146331686784 deprecation.py:339] From /usr/local/lib/python3.6/dist-packages/magenta/pipelines/pipeline.py:310: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager e

In [ ]:
%%shell
polyphony_rnn_train \
--run_dir=tmp/polyphony_rnn/logdir/run1 \
--sequence_example_file=tmp/polyphony_rnn/sequence_examples/training_poly_tracks.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[128,128,128]" \
--num_training_steps=200

2021-02-09 19:20:33.186822: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:hparams = {'batch_size': 64, 'rnn_layer_sizes': [128, 128, 128], 'dropout_keep_prob': 0.5, 'attn_length': 0, 'clip_norm': 5, 'learning_rate': 0.001, 'residual_connections': False, 'use_cudnn': False}
I0209 19:20:37.716249 140030526515072 events_rnn_graph.py:96] hparams = {'batch_size': 64, 'rnn_layer_sizes': [128, 128, 128], 'dropout_keep_prob': 0.5, 'attn_l